In [14]:
###### Receptive field size computation ######
import math;

########################################################### Calculator
class ReceptiveFieldCalculator():
    def calculate(self, architecture, input_image_size):
        input_layer = ('input_layer', input_image_size, 1, 1, 0.5)
        self._print_layer_info(input_layer)
        
        for key in architecture:
            current_layer = self._calculate_layer_info(architecture[key], input_layer, key)
            self._print_layer_info(current_layer)
            input_layer = current_layer

    def _print_layer_info(self, layer):
        print(f'------')
        print(f'{layer[0]}: n = {layer[1]}; stride = {layer[2]}; receptive field = {layer[3]}; start = {layer[4]}')     
        print(f'------')
            
    def _calculate_layer_info(self, current_layer, input_layer, layer_name):
        n_in = input_layer[1]
        j_in = input_layer[2]
        r_in = input_layer[3]
        start_in = input_layer[4]
        
        k = current_layer[0]
        s = current_layer[1]
        p = current_layer[2]

        n_out = math.floor((n_in - k + 2*p)/s) + 1
        padding = (n_out-1)*s - n_in + k 
        p_right = math.ceil(padding/2)
        p_left = math.floor(padding/2)

        j_out = j_in * s
        r_out = r_in + (k - 1)*j_in
        start_out = start_in + ((k-1)/2 - p_left)*j_in
        return layer_name, n_out, j_out, r_out, start_out
##########################################################


layers = {'conv1' : [3,1,1] ,
    'conv2' : [3,1,1],
    'maxpool1' : [2,2,0],
    'conv3' : [3,1,1] ,
    'conv4' : [3,1,1],
    'maxpool2' : [2,2,0],
    'conv5' : [3,1,1] ,
    'conv6' : [3,1,1],
    'conv7' : [3,1,1],
    'maxpool3' : [2,2,0],
    'conv8' : [3,1,1] ,
    'conv9' : [3,1,1],
    'conv10' : [3,1,1],
    'maxpool4' : [2,2,0],
    'conv11' : [3,1,1] ,
    'conv12' : [3,1,1],
    'conv13' : [3,1,1],
    'maxpool5' : [2,2,0]} # {'layer name' : [kernel, stride, padding]} format

calculator = ReceptiveFieldCalculator()
calculator.calculate(layers, 224)

------
input_layer: n = 224; stride = 1; receptive field = 1; start = 0.5
------
------
conv1: n = 224; stride = 1; receptive field = 3; start = 0.5
------
------
conv2: n = 224; stride = 1; receptive field = 5; start = 0.5
------
------
maxpool1: n = 112; stride = 2; receptive field = 6; start = 1.0
------
------
conv3: n = 112; stride = 2; receptive field = 10; start = 1.0
------
------
conv4: n = 112; stride = 2; receptive field = 14; start = 1.0
------
------
maxpool2: n = 56; stride = 4; receptive field = 16; start = 2.0
------
------
conv5: n = 56; stride = 4; receptive field = 24; start = 2.0
------
------
conv6: n = 56; stride = 4; receptive field = 32; start = 2.0
------
------
conv7: n = 56; stride = 4; receptive field = 40; start = 2.0
------
------
maxpool3: n = 28; stride = 8; receptive field = 44; start = 4.0
------
------
conv8: n = 28; stride = 8; receptive field = 60; start = 4.0
------
------
conv9: n = 28; stride = 8; receptive field = 76; start = 4.0
------
------
c